<a href="https://colab.research.google.com/github/akutayaydin/Magnimind-5.1-DeepLearning/blob/main/0_WarmUp1CNNExample1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN Example 1
For this example, we have images of cars and flowers, which have been divided into training and testing sets, and we have to build a CNN that identifies whether an image is a car or a flower.

### Step 1: Import the numpy library and the necessary Keras libraries and classes

In [2]:
# Import the Libraries
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np
from tensorflow import random

### Step 2: Now, set a seed and initiate the model with the `Sequential` class

In [3]:
#set a seed
seed = 1
np.random.seed(seed)
random.set_seed(seed)

# Initialising the CNN
classifier = Sequential()

### Step 3: Add the first layer of the CNN, set the input shape to (64, 64, 3), the dimension of each image, and set the activation function as a ReLU:

In [4]:
classifier.add(Conv2D(10,kernel_size =2, activation = 'relu',input_shape = (64,64,3)))

### Step 4: Now, add the pooling layer with the image size as 2x2

In [5]:
classifier.add(MaxPool2D(pool_size =(2,2)))

### Step 5: Flatten the output of the pooling layer by adding a flattening layer to the CNN model:

In [6]:
classifier.add(Flatten())

### Step 6: Add the first Dense layer of the MLP. 
Here, 128 is the output of the number of nodes. As a good practice, 128 is good to get started. activation is relu. As a good practice, the power of two is preferred

In [7]:
classifier.add(Dense(128,activation = 'relu'))

### Step 7: Add the output layer of the MLP.
This is a binary classification problem, so the size is 1 and the activation is `sigmoid`:

In [8]:
classifier.add(Dense(1,activation = 'sigmoid'))

### Step 8: Compile the network
Use an adam optimizer and compute the accuracy during the training process 

In [9]:
classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Step 9: Create training and test data generators. 
- Rescale the training and test images by `1/255` so that all the values are between `0` and `1`.
- Set these parameters for the training data generators only 
 - `shear_range=0.2`, `zoom_range=0.2`, and `horizontal_flip=True`
 
 - https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html


In [47]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [67]:
mypath='/content/gdrive/My Drive/Google Colab Folder/'
mypath_testdata = '/content/gdrive/My Drive/Google Colab Folder/train'
mypath_validationdata = '/content/gdrive/My Drive/Google Colab Folder/test'

In [69]:
#check for any file corruptions
import os
from PIL import Image

for name in os.listdir(mypath_testdata):
  dir = os.path.join(mypath_testdata, name)
  if os.path.isdir(dir):
    print('Dir name:' + str(name))
    for fname in os.listdir(dir):
      try:   
        img = Image.open(str(dir) + '/'+ str(fname))
        img.verify()  # verify that it is, in fact an image
      except:
        print('Bad file: ' + str(fname))

for name in os.listdir(mypath_validationdata):
  dir = os.path.join(mypath_validationdata, name)
  if os.path.isdir(dir):
    print('Dir name:' + str(name))
    for fname in os.listdir(dir):
      try:   
        img = Image.open(str(dir) + '/'+ str(fname))
        img.verify()  # verify that it is, in fact an image
      except:
        print('Bad file: ' + str(fname))

Dir name:car
Bad file: car1115.jpg
Dir name:flower
Dir name:car
Dir name:flower


In [70]:
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.utils import array_to_img, img_to_array, load_img

In [71]:
#Define batch size here
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale = 1. /255,
    shear_range =0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale = 1. /255)

### Step 10: Create a training set from the training set folder.
'training_set' is the folder where our data has been placed. Our CNN model has an image size of `64x64`, so the same size should be passed here too. `batch_size` is the number of images in a single batch, which is `32`. `Class_mode` is set to binary since we are working on binary classifiers

In [73]:
#Train data generation starts here
train_generator = train_datagen.flow_from_directory(
    directory = mypath_testdata,
    target_size = (64,64), #resize all images
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 1568 images belonging to 2 classes.


### Step 11: Repeat step 10 for the test set 
while setting the folder to the location of the test images, that is, 'test_set'

In [74]:
#Test data generation starts here
test_generator = test_datagen.flow_from_directory(
    directory = mypath_validationdata,
    target_size = (64,64),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 387 images belonging to 2 classes.


### Step 12: Finally, fit the data. 
Set the `steps_per_epoch` to `STEP_SIZE_TRAIN` and the `validation_steps` to `STEP_SIZE_TEST`. 

Why do we need `steps_per_epoch` ?

Keep in mind that a Keras data generator is meant to loop infinitely — it should never return or exit.

Since the function is intended to loop infinitely, Keras has no ability to determine when one epoch starts and a new epoch begins.

Therefore, we compute the `steps_per_epoch` value as the total number of training data points divided by the batch size. Once Keras hits this step count it knows that it’s a new epoch.

In [75]:
classifier.fit(
        train_generator,
        steps_per_epoch=49, #QUESTION??
        epochs=10,
        validation_data=test_generator,
        validation_steps=12) #QUESTION??

classifier.save_weights('/content/gdrive/My Drive/Google Colab Folder/first_try.h5')  

Epoch 1/10
49/49 [==============================] - 11s 209ms/step - loss: 0.6447 - accuracy: 0.6371 - val_loss: 0.5869 - val_accuracy: 0.6719
Epoch 2/10
49/49 [==============================] - 9s 182ms/step - loss: 0.5291 - accuracy: 0.7366 - val_loss: 0.5321 - val_accuracy: 0.7552
Epoch 3/10
49/49 [==============================] - 10s 211ms/step - loss: 0.4738 - accuracy: 0.7838 - val_loss: 0.4987 - val_accuracy: 0.7708
Epoch 4/10
49/49 [==============================] - 10s 204ms/step - loss: 0.4578 - accuracy: 0.7825 - val_loss: 0.4973 - val_accuracy: 0.7500
Epoch 5/10
49/49 [==============================] - 9s 180ms/step - loss: 0.4460 - accuracy: 0.7876 - val_loss: 0.4706 - val_accuracy: 0.7865
Epoch 6/10
49/49 [==============================] - 10s 206ms/step - loss: 0.4280 - accuracy: 0.7997 - val_loss: 0.4718 - val_accuracy: 0.7891
Epoch 7/10
49/49 [==============================] - 10s 212ms/step - loss: 0.4114 - accuracy: 0.8131 - val_loss: 0.4777 - val_accuracy: 0.7839
E